<h1 align="center"><font color="yellow">LangChain: Indexing API</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# <font color="red">Indexing API</font>

Links de estudo:

* [LangChain Indexing API - Update your Embeddings SURGICALLY!](https://www.youtube.com/watch?v=PvOb0gKMDzo)

* [GitHub: Vector Search with LangChain Indexing API](https://github.com/Coding-Crashkurse/LangChain-Indexing-API/tree/main)

In [1]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import TextLoader

embeddings = OpenAIEmbeddings()

import psycopg2

# Constrói a string de conexão PGVector a partir dos parâmetros.
host= os.environ['DB_HOST']
port= os.environ['DB_PORT']
user= os.environ['DB_USER']
password= os.environ['DB_PASSWORD']
dbname= os.environ['DB_NAME']

CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
COLLECTION_NAME = "Eddy_vectordb"


vectorstore = PGVector.from_documents(
    [],
    embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)


Vamos adicionar documentos e Embeddings!

In [3]:
loader = TextLoader("./bella_vista.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=150,
                                      chunk_overlap=20
                                     )


docs = text_splitter.split_documents(documents)
print(len(docs))

Created a chunk of size 179, which is longer than the specified 150
Created a chunk of size 231, which is longer than the specified 150
Created a chunk of size 280, which is longer than the specified 150
Created a chunk of size 210, which is longer than the specified 150
Created a chunk of size 231, which is longer than the specified 150
Created a chunk of size 317, which is longer than the specified 150


7


In [4]:
docs

[Document(page_content='Q: Eddy, Qual é o horário de funcionamento do Bella Vista?\nA: O Bella Vista funciona das 11h às 23h. de segunda a Sábado. Aos domingos, recebemos hóspedes a partir das 12h. às 22h', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Que tipo de culinária o Bella Vista serve?\nA: Bella Vista oferece uma deliciosa mistura de cozinha mediterrânea e americana contemporânea. Orgulhamo-nos de usar os ingredientes mais frescos, muitos dos quais são de origem local.', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Vocês oferecem opções vegetarianas ou veganas no Bella Vista?\nA: Absolutamente! Bella Vista possui um cardápio diversificado que inclui uma variedade de pratos vegetarianos e veganos. Nossos chefs também terão prazer em personalizar pratos com base nas necessidades dietéticas.', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Bella Vista é ideal para famílias?\nA: Sim, o Bella Vista é um estabel

In [5]:
from langchain.indexes import SQLRecordManager

# Atualize o namespace para refletir PGVector:
namespace = f"pgvector/{COLLECTION_NAME}"

record_manager = SQLRecordManager(namespace,
                                  db_url=CONNECTION_STRING
                                 )


# Criar schema para o Gerenciador de registros (record):
record_manager.create_schema()


<font color="orange">Atualize os documentos para ver algumas mudanças:</font>

In [ ]:
# loader = TextLoader("./bella_vista.txt")

# documents = loader.load()

# text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=20)
# docs = text_splitter.split_documents(documents)


# for doc in docs:
#     print(doc)

In [6]:
from langchain.indexes import index

index(
    docs,
    record_manager,
    vectorstore,
    #cleanup=None,
    delete_mode=None,
    source_id_key="source",
)


{'num_added': 7, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [7]:
from langchain.schema import Document

# A seguir fazemos três MUDANÇAS no Dataset:
docs[0].page_content = "Q: Quem é o secretário do Ally?\nA: É o Josemar"

del docs[1]

docs.append(Document(page_content="new content", metadata={"source": "important"}))


In [8]:
docs

[Document(page_content='Q: Quem é o secretário do Ally?\nA: É o Josemar', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Vocês oferecem opções vegetarianas ou veganas no Bella Vista?\nA: Absolutamente! Bella Vista possui um cardápio diversificado que inclui uma variedade de pratos vegetarianos e veganos. Nossos chefs também terão prazer em personalizar pratos com base nas necessidades dietéticas.', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Bella Vista é ideal para famílias?\nA: Sim, o Bella Vista é um estabelecimento familiar. Temos um menu infantil dedicado e oferecemos cadeiras altas e assentos elevatórios para os nossos hóspedes mais jovens.', metadata={'source': './bella_vista.txt'}),
 Document(page_content='Q: Posso reservar eventos privados no Bella Vista?\nA: Certamente! Bella Vista possui uma área de jantar privativa perfeita para eventos, festas ou reuniões corporativas. Também oferecemos serviços de catering para eventos exte

In [9]:
index(
    docs,
    record_manager,
    vectorstore,
    #cleanup=None,
    delete_mode=None,
    source_id_key="source",
)


{'num_added': 2, 'num_updated': 0, 'num_skipped': 5, 'num_deleted': 0}

In [10]:
index(
    docs,
    record_manager,
    vectorstore,
    #cleanup="incremental",
    delete_mode="incremental",
    source_id_key="source",
)


{'num_added': 0, 'num_updated': 0, 'num_skipped': 7, 'num_deleted': 2}

In [11]:
vectorstore.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.80,
                               'k':5}).get_relevant_documents("Quem é o secretário?")


[Document(page_content='Q: Quem é o secretário do Ally?\nA: É o Josemar', metadata={'source': './bella_vista.txt'})]

In [12]:
docs[0].page_content = "Q: Quem é o secretário?\nA: É Eddy"


In [13]:
index(docs,
      record_manager,
      vectorstore,
      #cleanup="full",
      delete_mode='incremental',
      source_id_key="source"
     )


{'num_added': 1, 'num_updated': 0, 'num_skipped': 6, 'num_deleted': 1}

In [ ]:
del docs[5]